This notebook can be run on the Copernicus Dataspace Jupyterhub but running the following package installation cell first

In [1]:
!pip install eomaji@git+https://github.com/DHI/EOMAJI-OpenEO-toolbox.git

# Data Mining Sharpener Workflow

In [3]:
import openeo
import rioxarray
from eomaji.workflows.decision_tree_sharpener import run_decision_tree_sharpener
#from eomaji.utils.draw_utils import draw_aoi

### Set up the OpenEO connection

In [8]:
connection = openeo.connect("https://openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

### Select the area of interest, the date and the collections you want to use and load the datacubes

In [13]:
date = "2023-06-11"
bbox = [8.16, 56.48, 8.24,  56.53] 
high_resolution_collection = "SENTINEL2_L2A"
low_resolution_collection = "SENTINEL3_SLSTR_L2_LST"
low_resolution_band = ["LST"]
low_resolution_mask_band = ["confidence_in"] # The low resolution mask is optional

In [14]:
aoi = dict(zip(["west", "south", "east", "north"], bbox))
# Load high-resolution collection
high_resolution_cube = connection.load_collection(
    high_resolution_collection,
    spatial_extent=aoi,
    temporal_extent=date,
)

# Load low-resolution collection
low_resolution_cube = connection.load_collection(
    low_resolution_collection,
    spatial_extent=aoi,
    temporal_extent=date,
    bands=low_resolution_band,
    properties={"timeliness": lambda x: x=="NT",
            "orbitDirection": lambda x: x=="DESCENDING"}
)

# Cloud mask layer
mask_cube = connection.load_collection(
    low_resolution_collection,
    temporal_extent=date,
    spatial_extent=aoi,
    bands=low_resolution_mask_band,
    properties={"timeliness": lambda x: x=="NT",
                "orbitDirection": lambda x: x=="DESCENDING"}
)
mask_cube_binary = mask_cube.apply(lambda x: (x < 16384)*1.0)

/usr/local/lib/python3.10/dist-packages/openeo/rest/connection.py:1319: UserWarning: SENTINEL3_SLSTR_L2_LST property filtering with properties that are undefined in the collection metadata (summaries): timeliness, orbitDirection.
  return DataCube.load_collection(


### Run Data Mining Sharpener

In [15]:
sharpened_data = run_decision_tree_sharpener(
    high_resolution_cube = high_resolution_cube,
    low_resolution_cube = low_resolution_cube,
    low_resolution_mask = mask_cube_binary, # The low resolution mask cube is optional, defaults to None
    mask_values = [1],
    cv_homogeneity_threshold = 0,
    moving_window_size = 30,
    disaggregating_temperature = True,
    n_jobs = 3,
    n_estimators = 30,
    max_samples = 0.8,
    max_features = 0.8,
)

0
Saved MEM


Warning 1: /tmp/tmpd32evip2.tiff: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.
Warning 1: /tmp/tmpd32evip2.tiff: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


In [16]:
sharpened_data

<xarray.Dataset> Size: 1MB
Dimensions:    (band: 1, y: 563, x: 500)
Coordinates:
  * x          (x) float64 4kB 4.483e+05 4.483e+05 ... 4.532e+05 4.532e+05
  * y          (y) float64 5kB 6.265e+06 6.265e+06 ... 6.26e+06 6.26e+06
  * band       (band) int64 8B 1
Data variables:
    band_data  (band, y, x) float32 1MB nan nan nan nan nan ... nan nan nan nan
Attributes:
    crs:        PROJCS["WGS 84 / UTM zone 32N",GEOGCS["WGS 84",DATUM["WGS_198...
    transform:  (448260.0, 10.0, 0.0, 6265390.0, 0.0, -10.0)

### Save data or continue the aggregations

In [18]:
sharpened_data.band_data.rio.to_raster('sharpened_LST.tif')